In [1]:
#Import
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import DataLoader
from torchsummary import summary
from torchvision.models import efficientnet_b0

from utils.dataset import BirdDataset
from utils.models import VisionTransformer, AutoEncoderLSTM, AutoEncoderAtt
from utils.utils import mixup

import matplotlib.pyplot as plt
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torchmetrics as tm
import wandb

/home/utilisateur/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/utilisateur/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/utilisateur/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
# Weight and biases
wandb.login(key = '15b11b57c09bdcd801af92cecb362a1f2634d213')
wandb.init(project="cuitcuit")
wandb_logger = WandbLogger()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nathan-vidal. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/utilisateur/.netrc


/home/utilisateur/.local/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


In [3]:
class Lumiere(pl.LightningModule):
    def __init__(self, model : nn.Module, criterion : nn.Module, num_classes : int, mixup : bool) -> None:
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.mixup = mixup
        self.num_classes = num_classes
        
        # logger
        self.save_hyperparameters()
        
        # metrics
        self.accuracy = tm.Accuracy(task = 'multiclass', num_classes = num_classes)
        self.f1 = tm.F1Score(task = 'multiclass', num_classes = num_classes)
        self.cf = tm.ConfusionMatrix(task = 'multiclass', num_classes = num_classes)

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        
        if self.mixup :
            y = torch.nn.functional.one_hot(y, self.num_classes)
            x, y = mixup(x, y, 0.5)
        
        logits = self.model(x)
        loss = self.criterion(logits, y)
        
        # metrics
        self.accuracy(logits, y)
        self.f1(logits, y)
        
        # log metrics
        self.log('train_loss', loss, prog_bar=True, logger = True, on_epoch=True)
        self.log('train_acc', self.accuracy, prog_bar=True, logger = True, on_epoch=True)
        self.log('train_f1', self.f1, prog_bar=True, logger = True, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # validation_step defines the validation loop.
        x, y = batch
            
        
        logits = self.model(x)
        loss = self.criterion(logits, y)
        
        # metrics
        self.accuracy(logits, y)
        self.f1(logits, y)
        
        # log metrics
        self.log('valid_loss', loss, prog_bar=True, logger = True, on_epoch=True)
        self.log('valid_acc', self.accuracy, prog_bar=True, logger = True, on_epoch=True)
        self.log('valid_f1', self.f1, prog_bar=True, logger = True, on_epoch=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        # test_step defines the test loop.
        x, y = batch
        logits = self.model(x)
        loss = self.criterion(logits, y)
        
        # metrics
        self.accuracy(logits, y)
        self.f1(logits, y)
        self.cf(logits, y)
        
        # log metrics
        self.log('test_loss', loss, prog_bar=True, logger = True, on_epoch=True)
        self.log('test_acc', self.accuracy, prog_bar=True, logger = True, on_epoch=True)
        self.log('test_f1', self.f1, prog_bar=True, logger = True, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 100, eta_min = 1e-6)
        return [optimizer], [lr_scheduler]

In [4]:
# Data
num_classes = 10
csv = pd.read_csv("dataset/spec.csv")

counts = csv['en'].value_counts()
chosen = counts[counts>=100].head(num_classes).index
csv = csv[csv['en'].isin(chosen)]

csv = csv.sample(frac = 1).reset_index(drop = True)
class_mapping = {bird : i for i, bird in enumerate(chosen)}
print(class_mapping)
train = csv.iloc[:int(len(csv)*0.8)]
valid = csv.iloc[int(len(csv)*0.8):int(len(csv)*0.9)]
test = csv.iloc[int(len(csv)*0.9):]

train_dataset = BirdDataset(train, "dataset/spectrograms/", class_mapping)
print(train_dataset[0][0].shape)
valid_dataset = BirdDataset(valid, "dataset/spectrograms/", class_mapping)
test_dataset = BirdDataset(test, "dataset/spectrograms/", class_mapping)

train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True, num_workers = 4, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size = 16, shuffle = False, num_workers = 4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size = 16, shuffle = False, num_workers = 4, pin_memory=True)

{'houwre': 0, 'houspa': 1, 'comrav': 2, 'eursta': 3, 'redcro': 4, 'gbwwre1': 5, 'sonspa': 6, 'spotow': 7, 'barswa': 8, 'norcar': 9}
torch.Size([3, 128, 313])


In [5]:
examples = next(iter(train_loader))
print(examples[0].shape)
# for label, img  in enumerate(examples):
#    plt.imshow(img[0,0,:,:])
#    plt.show()
#    print(f"Label: {label}")

torch.Size([16, 3, 128, 313])


In [6]:
## Model AutoEncoder using LSTM
modellstm = efficientnet_b0()
modellstm.classifier = nn.Identity()
aelstm = AutoEncoderLSTM(modellstm, 128, num_classes)

In [7]:
## Model AutoEncoder using TransformerDecoder ie Attention
modelatt = efficientnet_b0()
modelatt.classifier = nn.Identity()
aeatt = AutoEncoderAtt(modelatt, 1280, 10)

In [8]:
mixup_bool = True
model = efficientnet_b0()
model.classifier = nn.Sequential(
          nn.Dropout(p=0.2, inplace=True),
          nn.Linear(in_features=model.classifier[1].in_features, out_features = num_classes, bias = True)
      )
if mixup_bool :
    criterion = nn.BCEWithLogitsLoss()
else :
    criterion = nn.CrossEntropyLoss()
model_pl = Lumiere(aeatt, criterion, len(chosen), mixup_bool)
callbacks = [ModelCheckpoint(dirpath = 'save/lstm', monitor = 'valid_loss', save_top_k = 1, mode = 'min'), 
             EarlyStopping(monitor = 'valid_loss', patience = 2, mode = 'min')]

/home/utilisateur/.local/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
/home/utilisateur/.local/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion'])`.
  rank_zero_warn(


In [9]:
#training
trainer = Trainer(accelerator = 'gpu',
                  devices = 1,
                  max_epochs = 30,
                  logger = wandb_logger,
                  callbacks= callbacks,)

GPU available: True (cuda), used: True
/home/utilisateur/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/utilisateur/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(model = model_pl, 
             train_dataloaders = train_loader, 
             val_dataloaders = valid_loader)

/home/utilisateur/.local/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory /home/utilisateur/createch/tidzam/save/lstm exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                      | Params
--------------------------------------------------------
0 | model     | AutoEncoderAtt            | 114 M 
1 | criterion | CrossEntropyLoss          | 0     
2 | accuracy  | MulticlassAccuracy        | 0     
3 | f1        | MulticlassF1Score         | 0     
4 | cf        | MulticlassConfusionMatrix | 0     
--------------------------------------------------------
114 M     Trainable params
0         Non-trainable params
114 M     Total params
456.993   Total estimated model params size (MB)


Epoch 0: 100%|██████████| 1175/1175 [01:55<00:00, 10.17it/s, loss=2.3, v_num=7mw5, train_loss_step=2.260, train_acc_step=0.0833, train_f1_step=0.0833, valid_loss=2.340, valid_acc=0.106, valid_f1=0.106, train_loss_epoch=2.440, train_acc_epoch=0.000, train_f1_epoch=0.000]

/home/utilisateur/.local/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: The ``compute`` method of metric MulticlassAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
/home/utilisateur/.local/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: The ``compute`` method of metric MulticlassF1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


Epoch 8: 100%|██████████| 1175/1175 [01:56<00:00, 10.11it/s, loss=2.29, v_num=7mw5, train_loss_step=2.270, train_acc_step=0.0833, train_f1_step=0.0833, valid_loss=2.300, valid_acc=0.113, valid_f1=0.113, train_loss_epoch=2.300, train_acc_epoch=0.000, train_f1_epoch=0.000]


In [12]:
trainer.test(model_pl, dataloaders = test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 131/131 [00:02<00:00, 51.72it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.10823754966259003    │
│          test_f1          │    0.10823754966259003    │
│         test_loss         │    2.2984979152679443     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2.2984979152679443,
  'test_acc': 0.10823754966259003,
  'test_f1': 0.10823754966259003}]